In [ ]:
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | tee /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install google-chrome-stable

OK
deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,598 kB]
Get:13 http://archive.ubuntu.com/ubuntu

In [ ]:
!pip install selenium webdriver-manager beautifulsoup4

In [ ]:
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def extract_stock_data(html):
    """HTMLから株価データを解析してリストで返す"""
    soup = BeautifulSoup(html, 'html.parser')

# ユーザー指定のセレクタを使用してtdタグをすべて取得
    tds = soup.select('div.highcharts-tooltip > span > table > tbody > tr > td')

    # データが不足している場合はスキップ
    if len(tds) < 9:
        return None

    # tdタグからテキストを抽出
    # 通常、index 0:日付, 2:始値, 4:高値, 6:安値, 8:終値 の順になります
    raw_date  = tds[0].get_text(strip=True)
    val_open  = tds[1].get_text(strip=True)
    val_high  = tds[2].get_text(strip=True)
    val_low   = tds[3].get_text(strip=True)
    val_close = tds[4].get_text(strip=True)

# 日付の整形（例: "2024/5/8(水)" -> "2024-05-08"）
    try:
        # 曜日部分などを除外して数字とスラッシュのみ抽出
        clean_date = raw_date.split('(')[0]
        formatted_date = datetime.strptime(clean_date, '%Y/%m/%d').strftime('%Y-%m-%d')
    except Exception:
        formatted_date = raw_date

    return [formatted_date, val_open, val_high, val_low, val_close]

def get_stock_values(driver, url):
    """マウスホバーを利用して株価データを走査・取得する"""
    driver.get(url)
    # グラフの描画待ちをWebDriverWaitで置き換え

    # グラフ要素の取得
    try:
        wait = WebDriverWait(driver, 30) # 最大10秒待機
        graph_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.m-trend_chart_graph')))
    except Exception as e:
        print(f"Error finding graph element: {e}")
        return [] # 要素が見つからない場合は空のリストを返す

    width = graph_element.size['width']

    actions = ActionChains(driver)
    stock_data_list = []
    seen_dates = set()

    # 1. グラフの中央に移動し、そこから右端（幅の半分）へ移動
    # 2. 1pxずつ左へ移動しながらデータを抽出
    for offset in range(int(width / 2), -int(width / 2), -1):
        actions.move_to_element_with_offset(graph_element, offset, 0).perform()

        # 現在の表示状態のHTMLを取得して解析
        current_data = extract_stock_data(driver.page_source)

        if current_data and current_data[0] not in seen_dates:
            stock_data_list.append(current_data)
            seen_dates.add(current_data[0])

    return stock_data_list

if __name__ == "__main__":
    # Chromeの設定
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    target_url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"

    print("スクレイピングを開始")
    start_time = time.time() # 開始時間

    try:
        results = get_stock_values(driver, target_url)

        end_time = time.time() # 終了時間
        duration = end_time - start_time

        # 結果の表示
        print(f"\n--- 取得結果 (合計: {len(results)}件) ---")
        if results:
            print("日付 | 始値 | 高値 | 安値 | 終値")
            for data in results:
                print(" | ".join(data))
        else:
            print("データを取得できませんでした。ウェブサイトの構造が変更された可能性があります。")

        print(f"\n✅ スクレイピング完了")
        print(f"実行時間: {duration:.2f} 秒")

    finally:
        driver.quit()

スクレイピングを開始します... 🚀

--- 取得結果 (合計: 110件) ---
日付 | 始値 | 高値 | 安値 | 終値
2025-12-30 | 始値: 50,312.85 | 高値: 50,534.64 | 安値: 50,198.97 | 終値: 50,339.48
2025-12-29 | 始値: 50,691.22 | 高値: 50,707.23 | 安値: 50,347.65 | 終値: 50,526.92
2025-12-26 | 始値: 50,527.13 | 高値: 50,941.89 | 安値: 50,527.13 | 終値: 50,750.39
2025-12-25 | 始値: 50,450.18 | 高値: 50,510.11 | 安値: 50,283.76 | 終値: 50,407.79
2025-12-24 | 始値: 50,475.48 | 高値: 50,636.95 | 安値: 50,323.92 | 終値: 50,344.1
2025-12-23 | 始値: 50,374.48 | 高値: 50,544.56 | 安値: 50,264.42 | 終値: 50,412.87
2025-12-22 | 始値: 50,084.55 | 高値: 50,590.88 | 安値: 49,982.2 | 終値: 50,402.39
2025-12-19 | 始値: 49,387.71 | 高値: 49,766.96 | 安値: 49,257.15 | 終値: 49,507.21
2025-12-18 | 始値: 49,058.31 | 高値: 49,169.79 | 安値: 48,643.78 | 終値: 49,001.5
2025-12-17 | 始値: 49,413.19 | 高値: 49,571.5 | 安値: 49,077.81 | 終値: 49,512.28
2025-12-16 | 始値: 50,051.24 | 高値: 50,051.24 | 安値: 49,355.87 | 終値: 49,383.29
2025-12-15 | 始値: 50,352.09 | 高値: 50,432.1 | 安値: 49,965.68 | 終値: 50,168.11
2025-12-12 | 始値: 50,480.23 | 高値: 51,12